# Lib import

S&P 500 index is a market-capitalization index. This project will show how many shares of each constituents should I trade to get S&P 500 index fund. 

In [1]:
import pandas as pd
import numpy as np
import requests 
import xlsxwriter
import math

# Stock list

Below is the constituents list. The url for this list is given: https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view

In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]

# API token

I used IEX cloud API token, which will provide the data for me. This is a sandbox mode API token which is free. 

In [ ]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

# Call API 

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

# Parsing API call

In [ ]:
price = data['latestPrice']
market_cap = data['marketCap']
marketCap = float(market_cap)

# Creating a DataFrame using pandas 

In [ ]:
my_columns = ['Ticker','Stock Price', 'Market Capitalization']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

# Batch API for good performance

In [ ]:
def chunks(lst,n):
    """Yield successive n-siozed chunks from lst."""
    for i in range(0,len(lst),n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0,len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    print(batch_api_call_url)
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap']
                
            ],
            index = my_columns
         ),
        ignore_index = True
    
    
    
    )
final_dataframe



# Since the constituents has changed, I subbed out the N/A for 0

In [ ]:
marketCap_list = list(final_dataframe['Market Capitalization'])
type(marketCap_list)

replaced_marketCap_list = [0 if i is None else i for i in marketCap_list]
replaced_marketCap_list

# Getting portfolio size information 

In [ ]:
portfolio_size = input("Enter the value of your portfolio:")
val = float(portfolio_size)

# Converting Market-cap of each constituent company to a percentage of the Market-cap for all constituents. & add a column to dataframe &calculate position to buy for each constituents. 

In [ ]:
percentage_filtered_marketCap_list = 
for i in replaced_marketCap_list:
    percentage_filtered_marketCap_list.append(i/sum(replaced_marketCap_list))
    
final_dataframe[i,'Market Capitalization Percentage'] = percentage_filtered_marketCap_list
    
Position_to_Buy = []
for i in percentage_filtered_marketCap_list:
    Position_to_Buy.append(i * val)
    
final_dataframe[i,'Position to buy'] = Position_to_Buy

final_dataframe